# 🐕 Chihuahua vs 🧁 Muffin Classification - Hackathon Starter Notebook

## Quick Start Guide

Welcome to the **3LC x AWS Cloud Club @ UT Dallas Hackathon**! This notebook guides you through training an image classifier using **3LC** for data-centric AI.

**📋 Prerequisites:** Before running this notebook, complete the setup instructions in **Cell 1** below (scroll down).

### What You'll Learn:
1. **Environment Setup** - Verify and configure your environment
2. **Dataset Registration** - Create 3LC Tables for data versioning
3. **Model Training** - Train ResNet-18 with automatic experiment tracking
4. **Metrics Collection** - Analyze per-sample performance and embeddings
5. **Iterative Improvement** - Use 3LC Dashboard to improve data quality

### About 3LC (Three Lines of Code)
3LC is a data-centric AI platform that enables the **train-fix-retrain loop**:
- **Train** - Track experiments automatically
- **Analyze** - Use Dashboard to find data issues  
- **Fix** - Correct labels and improve quality
- **Retrain** - Iterate with better data

This workflow is how production AI teams achieve high accuracy with limited data!

### Dataset: Chihuahua vs Muffin  
Can you tell them apart? Your model will learn to classify:
- 🐕 **Chihuahua** - Small dog breed
- 🧁 **Muffin** - Baked good (surprisingly similar!)
- ❓ **Undefined** - Ambiguous cases for later labeling

### Notebook Structure:
This notebook combines the logic from these scripts:
- `register_tables.py` - Dataset registration with 3LC Tables
- `train.py` - Training ResNet-18 with full 3LC integration

**Let's begin!** 🚀


---
# Environment Setup Instructions

## Before You Begin

This notebook uses **3LC** for data-centric AI workflows. Follow these one-time setup steps:

### Step 0: Download the Dataset ⚠️

**IMPORTANT:** Before running this notebook, download the dataset using AWS CLI (no AWS account required).

---

#### A. Install AWS CLI (skip if already installed)

**Check if installed:**
Open a terminal and run:
```bash
aws --version
```

If you see a version number, skip to Step B. Otherwise, install it:

**Windows:**
1. Download the installer: [AWS CLI Windows Installer](https://awscli.amazonaws.com/AWSCLIV2.msi)
2. Run the installer (accept all defaults)
3. **Important:** Close and reopen your terminal/command prompt after installation
4. Verify: `aws --version`

**Mac:**
```bash
brew install awscli
```

**Linux:**
```bash
# Ubuntu/Debian
sudo apt install awscli

# RHEL/CentOS
sudo yum install aws-cli
```

---

#### B. Download the Dataset

Open a terminal/command prompt and navigate to the directory where you cloned this starter kit:

**Windows (PowerShell):**
```bash
cd C:\path\to\3lc-chihuahua-muffin-starter
aws s3 sync s3://3lc-hackathons/muffin-chihuahua/train128 ./train128 --no-sign-request
aws s3 sync s3://3lc-hackathons/muffin-chihuahua/test128 ./test128 --no-sign-request
```

**Mac/Linux:**
```bash
cd /path/to/3lc-chihuahua-muffin-starter
aws s3 sync s3://3lc-hackathons/muffin-chihuahua/train128 ./train128 --no-sign-request
aws s3 sync s3://3lc-hackathons/muffin-chihuahua/test128 ./test128 --no-sign-request
```

**What this does:**
- Downloads `train128/` and `test128/` folders directly into your current directory (no intermediate folders)
- The `--no-sign-request` flag means you don't need an AWS account
- Download size: ~150 MB (may take 2-5 minutes depending on internet speed)

---

#### C. Verify Download

After download completes, check that you have the correct structure:

**Windows:**
```powershell
ls train128
ls test128
```

**Mac/Linux:**
```bash
ls train128
ls test128
```

**Expected structure:**
```
3lc-chihuahua-muffin-starter/
├── train128/
│   ├── chihuahua/     (100 images)
│   ├── muffin/        (100 images)
│   └── undefined/     (4,533 unlabeled images)
├── test128/
│   ├── chihuahua/     (640 images)
│   └── muffin/        (544 images)
└── starter_notebook.ipynb (this file)
```

✅ **If you see these folders with images, you're ready to proceed!**

---

### Step 1: Create a 3LC Account (required for all teams and participants)
1. Go to [https://account.3lc.ai](https://account.3lc.ai)
2. Create your account (a workspace is auto-created for you)
3. Get your API key from [https://account.3lc.ai/api-key](https://account.3lc.ai/api-key)

   <div align="left">
   <img src="content/api.png" alt="Description" width="600">
   </div>

For Video tutorial refer [Here](https://docs.3lc.ai/3lc/latest/quickstart/quickstart.html)

### Step 2: Set Up Python Environment (activate every time you run notebook/scripts)

**Option A: Using venv (Recommended)**

Open a command prompt/terminal and run:

```bash
# Windows
python -m venv 3lc-env
3lc-env\Scripts\activate

# Linux/Mac
python -m venv 3lc-env
source 3lc-env/bin/activate

# Install 3LC (includes PyTorch, torchvision, and all dependencies)
pip install 3lc
pip install joblib pacmap
```

**Option B: Using Conda**
```bash
conda create -n 3lc-env python=3.10 -y
conda activate 3lc-env
pip install 3lc
pip install joblib pacmap
```

### Step 3: GPU Setup (Optional but Recommended)

If you have a GPU, reinstall PyTorch with CUDA support for 10x faster training:
```bash
# For CUDA 11.8
pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

# For CUDA 12.1
pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121
```
Visit [PyTorch.org](https://pytorch.org/get-started/locally/) to find the right command for your CUDA version.

### Step 4: Login to 3LC (Only required once)

Use the 3lc login command to connect your use of the package with your 3LC account workspace using an API key.
Open a command prompt/terminal (with your environment activated) and run:

```bash
3lc login <your_api_key>
```

Replace `<your_api_key>` with your actual API key from Step 1. This saves your credentials locally - you only need to do this once!

### Step 5: Start 3LC Service (Optional - Required for Dashboard visualization)

**Important:** The service is NOT required for training scripts to run, but it IS required to view results in the 3LC Dashboard.

We recommend starting it in the background. Open a **new/separate** command prompt/terminal window and type:

```bash
3lc service
```
   <div align="left">
   <img src="content/service.png" alt="Description" width="600">
   </div>

Keep this terminal window open while you work. The service can be stopped by pressing `Q`.

To view your results, open the Dashboard at [https://dashboard.3lc.ai](https://dashboard.3lc.ai)

---

### Quick Checklist:
- ✅ Dataset downloaded and placed in the correct location
- ✅ 3LC account created
- ✅ Python environment activated  
- ✅ Packages installed (`pip install 3lc`, `pip install joblib pacmap`)
- ✅ Logged in to 3LC (`3lc login <api_key>`)
- ✅ Service running (`3lc service`) - optional but recommended for Dashboard
- ✅ Kernel set to `3lc-env`

### 📝 Important Notes:

**For future sessions:** You only need to:
1. Activate your environment (Step 2)
2. Optionally start the service in a separate terminal (Step 5)

**Setting Kernel:** Make sure you are using the `3lc-env` kernel.:
- Click "Kernel" → "Change Kernel" → Select `3lc-env`

**Ready? Run the cells below in order!** 🚀


---
## Phase 1: Environment Verification & Dataset Registration

Let's verify your setup and register the dataset with 3LC.


In [2]:
# Imports 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import tlc
from tqdm import tqdm
from pathlib import Path
import sys

print(f"Python executable: {sys.executable}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Python executable: C:\Users\juhee\Desktop\Hackathon\3lc-env\Scripts\python.exe
Using device: cuda


In [3]:
# ============================================================================
# ENVIRONMENT VERIFICATION
# ============================================================================
print("Environment Check:")
print("=" * 70)
print(f"PyTorch version: {torch.__version__}")

# Check CUDA
if torch.cuda.is_available():
    print(f"[OK] CUDA available: {torch.cuda.get_device_name(0)}")
    print(f"     GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("[WARNING] CUDA not available - training will use CPU (slower)")

print(f"[OK] Using device: {device}")
print("=" * 70)


Environment Check:
PyTorch version: 2.5.1+cu121
[OK] CUDA available: NVIDIA GeForce RTX 4070
     GPU Memory: 12.0 GB
[OK] Using device: cuda


In [5]:
# ============================================================================
# DATASET VERIFICATION
# ============================================================================
print("\nDataset Check:")
print("=" * 70)

# Check for required directories
train_dir = Path("train128")
test_dir = Path("test128")
classes_to_check = ["chihuahua", "muffin", "undefined"]

if not train_dir.exists() or not test_dir.exists():
    print("[ERROR] Dataset not found!")
    print(f"  Expected 'train128' and 'test128' folders in: {Path('.').absolute()}")
    print("\n  Please download the dataset and place it in the same directory as this notebook.")
    print("  See Step 0 in the instructions above for details.")
    raise FileNotFoundError("Dataset directories not found")

# Count images in each split
total_train = 0
total_test = 0

for class_name in classes_to_check:
    train_class = train_dir / class_name
    test_class = test_dir / class_name
    
    if train_class.exists():
        train_count = len(list(train_class.glob('*.jpg')))
        total_train += train_count
        print(f"[OK] train128/{class_name}: {train_count} images")
    else:
        print(f"[WARNING] train128/{class_name}: folder not found")
    
    if test_class.exists():
        test_count = len(list(test_class.glob('*.jpg')))
        total_test += test_count
        print(f"[OK] test128/{class_name}: {test_count} images")
    else:
        print(f"[WARNING] test128/{class_name}: folder not found")

print(f"\n[OK] Total training images: {total_train}")
print(f"[OK] Total test images: {total_test}")
print("=" * 70)



Dataset Check:
[OK] train128/chihuahua: 100 images
[OK] test128/chihuahua: 640 images
[OK] train128/muffin: 100 images
[OK] test128/muffin: 544 images
[OK] train128/undefined: 4533 images
[WARNING] test128/undefined: folder not found

[OK] Total training images: 4733
[OK] Total test images: 1184


In [6]:
# ============================================================================
# DATASET REGISTRATION 
# ============================================================================

# Define constants
CLASSES = ["chihuahua", "muffin", "undefined"]
PROJECT_NAME = "Chihuahua-Muffin"
DATASET_NAME = "chihuahua-muffin"

# Define schemas
schemas = {
    "id": tlc.Schema(value=tlc.Int32Value(), writable=False),
    "image": tlc.ImagePath,
    "label": tlc.CategoricalLabel("label", classes=CLASSES),
    "weight": tlc.SampleWeightSchema(),
}

def register_dataset_to_table(dataset_path, table_name, split_name):
    """
    Register images from a folder structure to a 3LC table.
    EXACT function from register_tables.py
    """
    dataset_path = Path(dataset_path)
    image_data = []
    
    # Collect all images with their labels
    for class_idx, class_name in enumerate(CLASSES):
        class_folder = dataset_path / class_name
        if class_folder.exists():
            image_files = sorted(class_folder.glob('*.jpg'))
            print(f"Found {len(image_files)} images in {class_name} folder for {split_name} set")
            
            for img_path in image_files:
                image_data.append({
                    'path': str(img_path.absolute()),
                    'label': class_idx,
                })
        else:
            print(f"Warning: {class_folder} does not exist")
    
    print(f"\nTotal images for {split_name} set: {len(image_data)}")
    print(f"  - Chihuahua: {sum(1 for x in image_data if x['label'] == 0)}")
    print(f"  - Muffin: {sum(1 for x in image_data if x['label'] == 1)}")
    print(f"  - Undefined: {sum(1 for x in image_data if x['label'] == 2)}")
    
    # Create table writer
    table_writer = tlc.TableWriter(
        table_name=table_name,
        dataset_name=DATASET_NAME,
        project_name=PROJECT_NAME,
        description=f"Chihuahua vs Muffin {split_name} set with {len(image_data)} images",
        column_schemas=schemas,
        if_exists="overwrite",
    )
    
    # Add rows to the table
    for i, data in enumerate(image_data):
        label = data['label']
        weight = 1.0 if label in [0, 1] else 0.0
        
        table_writer.add_row({
            "id": i,
            "image": data['path'],
            "label": label,
            "weight": weight,
        })
    
    # Finalize the table
    table = table_writer.finalize()
    
    print(f"\n[OK] Created 3LC table: '{table_name}' with {len(image_data)} samples")
    print(f"  Table URL: {table.url}")
    
    return table

# Base path
base_path = Path(".")

print("=" * 60)
print("Registering Chihuahua vs Muffin Dataset in 3LC Tables")
print("=" * 60)

# Register train set
print("\n[1/2] Registering TRAIN set...")
print("-" * 60)
train_path = base_path / 'train128'
train_table = register_dataset_to_table(train_path, 'train', 'train')

# Register test set
print("\n[2/2] Registering TEST set...")
print("-" * 60)
test_path = base_path / 'test128'
test_table = register_dataset_to_table(test_path, 'test', 'test')

print("\n" + "=" * 60)
print("[OK] Successfully registered both tables!")
print("=" * 60)


Registering Chihuahua vs Muffin Dataset in 3LC Tables

[1/2] Registering TRAIN set...
------------------------------------------------------------
Found 100 images in chihuahua folder for train set
Found 100 images in muffin folder for train set
Found 4533 images in undefined folder for train set

Total images for train set: 4733
  - Chihuahua: 100
  - Muffin: 100
  - Undefined: 4533

[OK] Created 3LC table: 'train' with 4733 samples
  Table URL: C:/Users/juhee/AppData/Local/3LC/3LC/projects/Chihuahua-Muffin/datasets/chihuahua-muffin/tables/train

[2/2] Registering TEST set...
------------------------------------------------------------
Found 640 images in chihuahua folder for test set
Found 544 images in muffin folder for test set

Total images for test set: 1184
  - Chihuahua: 640
  - Muffin: 544
  - Undefined: 0

[OK] Created 3LC table: 'test' with 1184 samples
  Table URL: C:/Users/juhee/AppData/Local/3LC/3LC/projects/Chihuahua-Muffin/datasets/chihuahua-muffin/tables/test

[OK] Suc

---
## Phase 2: Model Definition & Training Setup

Now we'll define our ResNet-18 model and prepare for training.


In [14]:
# ============================================================================
# MODEL DEFINITION 
# ============================================================================

class ResNet18Classifier(nn.Module):
    """ResNet-18 model for classification (no pretrained weights)"""
    def __init__(self, num_classes=2):
        super(ResNet18Classifier, self).__init__()
        # Load ResNet-18 without pretrained weights
        self.resnet = models.resnet18(weights=None)
        
        # Get the number of features from ResNet's final layer
        resnet_features = self.resnet.fc.in_features
        
        # Remove the original final layer
        self.resnet.fc = nn.Identity()
        
        # Create new classification head
        self.classifier = nn.Sequential(
            nn.Linear(resnet_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        # Get ResNet features (without final classification layer)
        resnet_features = self.resnet(x)
        # Pass through classification head
        return self.classifier(resnet_features)

# Transforms 
train_transform = transforms.Compose([
    transforms.Resize(128),
    transforms.RandomCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(128),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

def train_fn(sample):
    """Transform function for training """
    image = Image.open(sample['image'])
    if image.mode != 'RGB':
        image = image.convert('RGB')
    return train_transform(image), sample['label']

def val_fn(sample):
    """Transform function for validation """
    image = Image.open(sample['image'])
    if image.mode != 'RGB':
        image = image.convert('RGB')
    return val_transform(image), sample['label']

print("[OK] Model and transforms defined")


[OK] Model and transforms defined


In [15]:
# ============================================================================
# METRICS FUNCTION 
# ============================================================================

def metrics_fn(batch, predictor_output: tlc.PredictorOutput):
    """Metrics function for 3LC collection"""
    labels = batch[1].to(device)
    predictions = predictor_output.forward
    
    # Softmax for probabilities
    softmax_output = F.softmax(predictions, dim=1)
    predicted_indices = torch.argmax(predictions, dim=1)
    confidence = torch.gather(softmax_output, 1, predicted_indices.unsqueeze(1)).squeeze(1)
    accuracy = (predicted_indices == labels).float()
    
    # Compute loss, set to 1.0 for labels outside valid range
    valid_labels = labels < predictions.shape[1]
    cross_entropy_loss = torch.ones_like(labels, dtype=torch.float32)
    cross_entropy_loss[valid_labels] = nn.CrossEntropyLoss(reduction="none")(
        predictions[valid_labels], labels[valid_labels]
    )
    
    return {
        "loss": cross_entropy_loss.cpu().numpy(),
        "predicted": predicted_indices.cpu().numpy(),
        "accuracy": accuracy.cpu().numpy(),
        "confidence": confidence.cpu().numpy(),
    }

print("[OK] Metrics function defined")


[OK] Metrics function defined


In [16]:
# ============================================================================
# TRAINING SETUP 
# ============================================================================

# Settings
EPOCHS = 10
BATCH_SIZE = 16
LEARNING_RATE = 0.001

print(f"Loaded table with {len(train_table)} samples")
class_names = list(train_table.get_simple_value_map("label").values())
print(class_names)

# Apply map functions 
train_table.map(train_fn).map_collect_metrics(val_fn)
test_table.map(val_fn)

# Create sampler 
sampler = train_table.create_sampler(exclude_zero_weights=True)

# Create dataloaders 
train_dataloader = DataLoader(
    train_table,
    batch_size=BATCH_SIZE,
    sampler=sampler,
    num_workers=0,  # Important for Windows compatibility
)

val_dataloader = DataLoader(
    test_table,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
)

# Initialize model 
model = ResNet18Classifier(num_classes=len(class_names)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Initialize 3LC run 
run = tlc.init(
    project_name=train_table.project_name,
    description="Finetuning classifier for active learning"
)

# Metric schemas 
metric_schemas = {
    "loss": tlc.Schema(
        description="Cross entropy loss",
        value=tlc.Float32Value(),
    ),
    "predicted": tlc.CategoricalLabelSchema(
        display_name="predicted label",
        classes=class_names,
    ),
    "accuracy": tlc.Schema(
        description="Per-sample accuracy",
        value=tlc.Float32Value(),
    ),
    "confidence": tlc.Schema(
        description="Prediction confidence",
        value=tlc.Float32Value(),
    ),
}

# Create metrics collector 
classification_metrics_collector = tlc.FunctionalMetricsCollector(
    collection_fn=metrics_fn,
    column_schemas=metric_schemas,
)

# Find embeddings layer 
indices_and_modules = list(enumerate(model.resnet.named_modules()))
resnet_fc_layer_index = None
for idx, (name, _) in indices_and_modules:
    if name == 'fc':
        resnet_fc_layer_index = idx
        break

if resnet_fc_layer_index is None:
    resnet_fc_layer_index = len(indices_and_modules) - 1

print(f"Using layer {resnet_fc_layer_index} for embeddings collection")

# Create embeddings collector and predictor 
embeddings_metrics_collector = tlc.EmbeddingsMetricsCollector(layers=[resnet_fc_layer_index])
predictor = tlc.Predictor(model, layers=[resnet_fc_layer_index])

print("[OK] Training setup complete")


Loaded table with 4733 samples
['chihuahua', 'muffin', 'undefined']
Using layer 67 for embeddings collection
[OK] Training setup complete


---
## Phase 3: Training Loop

Now we'll train the model and track everything with 3LC.


In [17]:
# ============================================================================
# TRAINING LOOP 
# ============================================================================

best_val_accuracy = 0.0
best_model_state = None

for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0.0
    
    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{EPOCHS}')
    for images, labels in progress_bar:
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        progress_bar.set_postfix({'loss': loss.item()})

    model.eval()
    # Run standard validation
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    val_progress_bar = tqdm(val_dataloader, desc=f'Validation Epoch {epoch+1}/{EPOCHS}')
    with torch.no_grad():
        for images, labels in val_progress_bar:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_correct += (outputs.argmax(1) == labels).sum().item()
            val_total += labels.size(0)
            val_progress_bar.set_postfix({'val_loss': loss.item()})
        
    # Calculate epoch metrics
    val_avg_loss = val_loss / len(val_dataloader)
    val_accuracy = 100 * val_correct / val_total
    
    print(f"Epoch {epoch+1}/{EPOCHS} - Val Loss: {val_avg_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
    
    # Save best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model_state = model.state_dict().copy()
        print(f"  [OK] New best model! Validation accuracy: {best_val_accuracy:.2f}%")
    
    # Log to 3LC
    tlc.log({
        "epoch": epoch,
        "val_loss": val_avg_loss,
        "val_accuracy": val_accuracy,
    })

print("\n" + "=" * 60)
print("Training complete!")
print(f"Best validation accuracy: {best_val_accuracy:.2f}%")
print("=" * 60)


Validation Epoch 1/10: 100%|██████████| 74/74 [00:02<00:00, 30.33it/s, val_loss=24.6]    


Epoch 1/10 - Val Loss: 13.3647, Val Accuracy: 54.05%
  [OK] New best model! Validation accuracy: 54.05%


Validation Epoch 2/10: 100%|██████████| 74/74 [00:02<00:00, 29.43it/s, val_loss=47.5]    


Epoch 2/10 - Val Loss: 23.4807, Val Accuracy: 56.25%
  [OK] New best model! Validation accuracy: 56.25%


Validation Epoch 3/10: 100%|██████████| 74/74 [00:02<00:00, 29.92it/s, val_loss=4.62]  


...

Epoch 8/10 - Val Loss: 0.4936, Val Accuracy: 80.41%


Validation Epoch 9/10: 100%|██████████| 74/74 [00:02<00:00, 28.62it/s, val_loss=0.587]  


Epoch 9/10 - Val Loss: 0.5880, Val Accuracy: 81.00%


Validation Epoch 10/10: 100%|██████████| 74/74 [00:02<00:00, 29.95it/s, val_loss=0.0345] 

Epoch 10/10 - Val Loss: 1.0829, Val Accuracy: 57.09%

Training complete!
Best validation accuracy: 82.94%


---
## Phase 4: Metrics Collection

Now we'll collect per-sample metrics using the best model.


In [18]:
# ============================================================================
# METRICS COLLECTION 
# ============================================================================

# Load best model
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print("\n[OK] Loaded best model for metrics collection")

# Save best model
torch.save(model.state_dict(), 'resnet18_classifier_best.pth')
print("[OK] Best model saved to 'resnet18_classifier_best.pth'")

# Collect metrics
print("\nCollecting metrics on train set with best model...")
model.eval()
tlc.collect_metrics(
    train_table,
    predictor=predictor,
    metrics_collectors=[classification_metrics_collector, embeddings_metrics_collector],
    split="train",
    dataloader_args={"batch_size": BATCH_SIZE, "num_workers": 0},
)

print("\n[OK] Metrics collection complete!")


Output()


[OK] Loaded best model for metrics collection
[OK] Best model saved to 'resnet18_classifier_best.pth'




[OK] Metrics collection complete!


---
## Phase 5: Embeddings Reduction

Finally, we'll reduce the embeddings for 3D visualization in the Dashboard.


In [19]:
# ============================================================================
# EMBEDDINGS REDUCTION 
# ============================================================================

print("\nReducing embeddings using PaCMAP...")
run.reduce_embeddings_by_foreign_table_url(
    train_table.url,
    method="pacmap",
    n_neighbors=2,
    n_components=3,
)
print("Embeddings reduction complete!")

run.set_status_completed()

print("\n" + "=" * 60)
print("[OK] Complete! View results at https://dashboard.3lc.ai")
print("=" * 60)



Reducing embeddings using PaCMAP...
Embeddings reduction complete!

[OK] Complete! View results at https://dashboard.3lc.ai


---
# 🎉 Training Complete!

## What You've Accomplished:
✅ **Registered dataset** with 3LC Tables for version control  
✅ **Trained ResNet-18** classifier with full 3LC integration  
✅ **Collected metrics** - per-sample loss, accuracy, confidence, embeddings  
✅ **Created 3LC Run** - visible in Dashboard with complete tracking  

## Next Steps: The Train-Fix-Retrain Loop

### 1. Analyze Your Results in 3LC Dashboard

Open [https://dashboard.3lc.ai](https://dashboard.3lc.ai) and navigate to your project: `Chihuahua-Muffin`

**What to Look For:**
- **Low-confidence predictions** - Model is uncertain filter embeddings
- **High-loss samples** - Model performs poorly  
- **Outliers in embeddings** - Potentially mislabeled or ambiguous
- **Class confusion patterns** - Which classes are confused?

### 2. Identify Data Quality Issues

Common issues to fix:
- **Wrong labels** - Chihuahua labeled as Muffin or vice versa
- **Poor quality images** - Blurry, corrupted, or irrelevant
- **Ambiguous cases** - Move to "undefined" class for later review

### 3. Fix Data in Dashboard

3LC Dashboard allows you to:
- **Edit labels** directly on images
- **Add/remove samples** from training set
- **Adjust sample weights** to focus on hard examples
- **Create filtered views** to find problematic samples

**Learn more:** [Edit Tables in Dashboard](https://docs.3lc.ai/3lc/latest/how-to/basics/edit-table.html)

### 4. Retrain with Improved Data

After editing in Dashboard:
1. The Dashboard creates a **new table version** automatically
2. **Rerun cells 5-15** (from dataset registration through embeddings)
3. 3LC automatically uses the latest version with your edits if you use the .latest() method
4. Compare runs in Dashboard to see improvement!

**Pro Tip:** Change the `description` parameter in `tlc.init()` (Cell 9) to track different iterations (e.g., "baseline_v1", "fixed_labels_v2")

### 5. Iterate for Maximum Performance

Each iteration should:
- Fix a specific type of error
- Document what you changed
- Measure improvement in validation accuracy
- Identify next bottleneck

**This iterative process is data-centric AI in action!**

---

## Resources & Support:

### Documentation:
- [3LC Documentation](https://docs.3lc.ai)
- [3LC Dashboard Guide](https://docs.3lc.ai/3lc/latest/user-guide/dashboard/)
- [PyTorch Documentation](https://pytorch.org/docs/)

### 3LC Dashboard:
- [Open Dashboard](https://dashboard.3lc.ai)
- View your project: `Chihuahua-Muffin`
- Explore runs, metrics, and embeddings

### Need Help?
- Check the documentation links above
- Ask hackathon organizers
- Review the example scripts (`register_tables.py`, `train.py`)

---

## Professional Skills You're Developing:

✅ **Data-Centric AI** - Using model feedback to improve data quality  
✅ **Experiment Tracking** - Systematic approach to model development  
✅ **Iterative Improvement** - Train-fix-retrain methodology  
✅ **Production Workflows** - How real AI teams work

**These are the skills that separate good ML engineers from great ones!**

---

**Ready to improve your model? Open the Dashboard and start analyzing!** 🚀
